# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'08-07-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'08-07-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-08-08 04:34:53,34.223334,-82.461707,321,8,0,313.0,"Abbeville, South Carolina, US",1308.761773,2.492212
1,22001.0,Acadia,Louisiana,US,2020-08-08 04:34:53,30.295065,-92.414197,2570,79,0,2491.0,"Acadia, Louisiana, US",4142.154888,3.073930
2,51001.0,Accomack,Virginia,US,2020-08-08 04:34:53,37.767072,-75.632346,1087,16,0,1071.0,"Accomack, Virginia, US",3363.658869,1.471941
3,16001.0,Ada,Idaho,US,2020-08-08 04:34:53,43.452658,-116.241552,8720,76,0,8644.0,"Ada, Idaho, US",1810.680106,0.871560
4,19001.0,Adair,Iowa,US,2020-08-08 04:34:53,41.330756,-94.471059,28,0,0,28.0,"Adair, Iowa, US",391.498881,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,36471,36542,36675,36710,36710,36747,36782,36829,36896,37015
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,5105,5197,5276,5396,5519,5620,5750,5889,6016,6151
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,29229,29831,30394,30950,31465,31972,32504,33055,33626,34155


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1271,1271,1272,1283,1284,1288,1288,1294,1298,1307
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,150,154,157,161,166,172,176,182,188,189
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1186,1200,1210,1223,1231,1239,1248,1261,1273,1282


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,25389,25471,25509,25509,25510,25669,25669,25742,25840,25903
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2830,2883,2952,2961,3018,3031,3031,3123,3155,3227
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,19592,20082,20537,20988,21419,21901,22375,22802,23238,23667


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,354,356,356,356,368,375,389,397,411,411
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,42,42,42,44,45,46,46,46,47,47


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
578,1029.0,Cleburne,Alabama,US,2020-08-08 04:34:53,33.676792,-85.520059,125,1,0,124.0,"Cleburne, Alabama, US",838.363514,0.800000
1386,1071.0,Jackson,Alabama,US,2020-08-08 04:34:53,34.781442,-85.997505,945,4,0,941.0,"Jackson, Alabama, US",1830.473017,0.423280
1779,1089.0,Madison,Alabama,US,2020-08-08 04:34:53,34.763271,-86.550696,5250,30,0,5220.0,"Madison, Alabama, US",1407.850173,0.571429


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,4941755,161347,1623870
Brazil,2962442,99572,2272299
India,2088611,42518,1427005


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,4941755,161347,1623870,3156538,2020-08-08 04:34:53,37.906189,-91.329468
Brazil,2962442,99572,2272299,590571,2020-08-08 04:34:53,-12.669522,-48.480493
India,2088611,42518,1427005,619088,2020-08-08 04:34:53,23.088275,81.806127
Russia,875378,14698,682278,178402,2020-08-08 04:34:53,54.546312,62.120860
South Africa,545476,9909,394759,140808,2020-08-08 04:34:53,-30.559500,22.937500


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,546814,10133,0
Florida,518075,7927,0
Texas,491304,8847,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,546814,10133,0,536681,2020-08-08 04:34:53,37.843962,-120.728594
Florida,518075,7927,0,510148,2020-08-08 04:34:53,28.940755,-82.700744
Texas,491304,8847,0,482457,2020-08-08 04:34:53,31.660643,-98.653069
New York,419642,32760,0,386882,2020-08-08 04:34:53,42.671593,-75.579694
Georgia,209004,4117,0,204887,2020-08-08 04:34:53,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,227258,23567,0
California,Los Angeles,204258,4919,0
Florida,Miami-Dade,129409,1809,0
Arizona,Maricopa,124924,2307,0
Illinois,Cook,110002,4920,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,227258,23567,0,203691,2020-08-08 04:34:53,40.767273,-73.971526,36061.0
California,Los Angeles,204258,4919,0,199339,2020-08-08 04:34:53,34.308284,-118.228241,6037.0
Florida,Miami-Dade,129409,1809,0,127600,2020-08-08 04:34:53,25.611236,-80.551706,12086.0
Arizona,Maricopa,124924,2307,0,122617,2020-08-08 04:34:53,33.348359,-112.491815,4013.0
Illinois,Cook,110002,4920,0,105082,2020-08-08 04:34:53,41.841448,-87.816588,17031.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-05,36829,5889,33055,939,1395,92,220682,39586,19890,21566,...,308832,1309,27793,22299,717,13065,10,1763,7022,4221
2020-08-06,36896,6016,33626,944,1483,92,228195,39819,20272,21696,...,309796,1318,28315,23280,750,13398,10,1768,7164,4339
2020-08-07,37015,6151,34155,955,1538,92,235677,39985,20698,21837,...,310696,1325,29057,24166,789,13722,10,1796,7486,4451


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-05,1294,182,1261,52,62,3,4106,770,255,719,...,46449,37,171,195,9,89,1,508,176,81
2020-08-06,1298,188,1273,52,64,3,4251,772,266,719,...,46498,37,175,202,10,92,1,508,199,84
2020-08-07,1307,189,1282,52,67,3,4411,777,278,720,...,46596,37,181,208,10,94,1,512,200,102


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-05,25742,3123,22802,825,506,76,96948,30850,10941,19464,...,1446,1065,18783,12146,381,6618,8,894,5667,1238
2020-08-06,25840,3155,23238,828,520,76,99852,31556,11119,19596,...,1447,1079,19291,12470,392,6907,8,898,5786,1264
2020-08-07,25903,3227,23667,839,544,76,103297,32008,11560,19690,...,1447,1095,20059,12470,395,7210,8,907,6264,1345


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20
130,84001097,US,USA,840,1097.0,Mobile,Alabama,US,30.784723,-88.208424,...,7683,7879,8084,8466,8998,9086,9170,9269,9420,9565
109,84001055,US,USA,840,1055.0,Etowah,Alabama,US,34.045673,-86.040519,...,1743,1797,1857,1884,1914,1930,1957,1967,2041,2075
110,84001057,US,USA,840,1057.0,Fayette,Alabama,US,33.720769,-87.738866,...,156,162,163,167,169,174,176,178,183,198
111,84001059,US,USA,840,1059.0,Franklin,Alabama,US,34.442353,-87.842895,...,1117,1144,1162,1165,1184,1196,1206,1215,1238,1258
112,84001061,US,USA,840,1061.0,Geneva,Alabama,US,31.093890,-85.835728,...,198,204,210,216,226,232,242,243,248,255


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-05,94654,3448,0,182203,46293,530606,48970,50225,15296,49,...,114098,476999,42328,1436,481,95049,60084,7162,56940,2923
2020-08-06,96592,3483,0,183656,47028,541339,49418,50245,15365,49,...,116350,483920,42915,1445,501,95867,60917,7278,57779,2953
2020-08-07,98301,3536,0,185053,48039,546814,49875,50320,15445,49,...,118782,491304,43375,1448,522,97882,61587,7452,58768,3000


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-05        1030    3322     564  384    745     444    754    1627   
2020-08-06        1050    3380     565  413    759     450    756    1672   
2020-08-07        1065    3443     569  416    774     454    759    1704   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-05          832      249  ...     124      5       64       39   
2020-08-06          835      258  ...     128      5       68       38   
2020-08-07          835      263  ...     132      5       69       38   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-05            252   363   268          0       58      5  
2020-08-06            253   367   272          0       59      5  
2020-08-07            257   371   274          0       69      5  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-05,1695,25,0,3932,508,9808,1851,4437,587,0,...,1144,8310,327,57,8,2274,1624,124,970,27
2020-08-06,1714,25,0,4002,515,10021,1852,4437,587,0,...,1186,8569,330,58,9,2299,1653,124,978,27
2020-08-07,1735,25,0,4081,521,10133,1857,4441,588,0,...,1206,8847,335,58,9,2317,1672,127,990,28


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-05          21      23       5    4      3      11     35      13   
2020-08-06          21      23       5    4      3      11     35      13   
2020-08-07          21      23       5    4      3      11     35      13   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-05           38        7  ...       0      0        0        0   
2020-08-06           38        7  ...       0      0        0        0   
2020-08-07           38        7  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-05              0     0     0         26        0      0  
2020-08-06              0     0     0         26        0      0  
2020-08-07              0     0     0         27        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-05,0.001278,0.024174,0.016952,0.000000,0.037946,0.0,0.033470,0.007329,0.022885,0.003957,...,0.002942,0.006923,0.027582,0.040162,0.066964,0.023101,0.0,0.001705,0.033711,0.000000
2020-08-06,0.001819,0.021566,0.017274,0.005325,0.063082,0.0,0.034044,0.005886,0.019206,0.006028,...,0.003121,0.006875,0.018782,0.043993,0.046025,0.025488,0.0,0.002836,0.020222,0.027955
2020-08-07,0.003225,0.022440,0.015732,0.011653,0.037087,0.0,0.032788,0.004169,0.021014,0.006499,...,0.002905,0.005311,0.026205,0.038058,0.052000,0.024183,0.0,0.015837,0.044947,0.025812


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-05,0.004658,0.034091,0.010417,0.0,0.050847,0.0,0.031918,0.002604,0.032389,0.000000,...,0.002915,0.0,0.036364,0.042781,0.125000,0.034884,0.0,0.003953,0.017341,0.000000
2020-08-06,0.003091,0.032967,0.009516,0.0,0.032258,0.0,0.035314,0.002597,0.043137,0.000000,...,0.001055,0.0,0.023392,0.035897,0.111111,0.033708,0.0,0.000000,0.130682,0.037037
2020-08-07,0.006934,0.005319,0.007070,0.0,0.046875,0.0,0.037638,0.006477,0.045113,0.001391,...,0.002108,0.0,0.034286,0.029703,0.000000,0.021739,0.0,0.007874,0.005025,0.214286


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-05,0.002844,0.030353,0.019084,0.000000,0.005964,0.013333,0.029948,0.015738,0.013149,0.006620,...,0.000692,0.016221,0.040552,0.022821,0.007937,0.031002,0.0,0.035921,0.109219,0.000000
2020-08-06,0.003807,0.010247,0.019121,0.003636,0.027668,0.000000,0.029954,0.022885,0.016269,0.006782,...,0.000692,0.013146,0.027046,0.026675,0.028871,0.043669,0.0,0.004474,0.020999,0.021002
2020-08-07,0.002438,0.022821,0.018461,0.013285,0.046154,0.000000,0.034501,0.014324,0.039662,0.004797,...,0.000000,0.014829,0.039811,0.000000,0.007653,0.043869,0.0,0.010022,0.082613,0.064082


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-05,0.010160,0.015910,NaN,0.009407,0.020097,0.007445,0.012279,0.002295,0.010504,0.0,...,0.014737,0.023533,0.010046,0.003494,0.038877,0.008467,0.011873,0.014591,0.015770,0.013523
2020-08-06,0.020475,0.010151,NaN,0.007975,0.015877,0.020228,0.009148,0.000398,0.004511,0.0,...,0.019737,0.014509,0.013868,0.006267,0.041580,0.008606,0.013864,0.016197,0.014735,0.010263
2020-08-07,0.017693,0.015217,NaN,0.007607,0.021498,0.010114,0.009248,0.001493,0.005207,0.0,...,0.020902,0.015259,0.010719,0.002076,0.041916,0.021019,0.010999,0.023908,0.017117,0.015916


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-05      0.000000  0.018082  0.001776  0.007874  0.005398  0.006803   
2020-08-06      0.019417  0.017459  0.001773  0.075521  0.018792  0.013514   
2020-08-07      0.014286  0.018639  0.007080  0.007264  0.019763  0.008889   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-08-05      0.003995  0.023270  0.004831  0.008097  ...  0.059829    0.0   
2020-08-06      0.002653  0.027658  0.003606  0.036145  ...  0.032258    0.0   
2020-08-07      0.003968  0.019139  0.000000  0.019380  ...  0.031250    0.0   

Province_State                                                                \
Admin2          Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2020-08-05      0.032258  0.000000   0.003984  0.002762  0.003745        NaN   
2020-08-06      0.062500 -0.025641   0.003968  0.011019  0.014925        NaN   
2020-08-07      0.014706  0.000000   0.015810  0.010899  0.007353        NaN   

Province_State                   
Admin2          Washakie Weston  
2020-08-05      0.074074    0.0  
2020-08-06      0.017241    0.0  
2020-08-07      0.169492    0.0  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-05,0.017407,0.0,NaN,0.022627,0.036735,0.012805,0.001082,0.000000,0.000000,NaN,...,0.024172,0.045020,0.018692,0.000000,0.000,0.013369,0.003088,0.000000,0.009365,0.000000
2020-08-06,0.011209,0.0,NaN,0.017803,0.013780,0.021717,0.000540,0.000000,0.000000,NaN,...,0.036713,0.031167,0.009174,0.017544,0.125,0.010994,0.017857,0.000000,0.008247,0.000000
2020-08-07,0.012252,0.0,NaN,0.019740,0.011650,0.011177,0.002700,0.000902,0.001704,NaN,...,0.016863,0.032443,0.015152,0.000000,0.000,0.007829,0.011494,0.024194,0.012270,0.037037


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                  \
Admin2         Autauga Baldwin Barbour      Bibb Blount Bullock Butler   
2020-08-05        0.05     0.0     0.0  0.333333    0.0     0.0    0.0   
2020-08-06        0.00     0.0     0.0  0.000000    0.0     0.0    0.0   
2020-08-07        0.00     0.0     0.0  0.000000    0.0     0.0    0.0   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2020-08-05      0.083333      0.0      0.0  ...     NaN    NaN      NaN   
2020-08-06      0.000000      0.0      0.0  ...     NaN    NaN      NaN   
2020-08-07      0.000000      0.0      0.0  ...     NaN    NaN      NaN   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-05          NaN        NaN   NaN   NaN   0.000000      NaN    NaN  
2020-08-06          NaN        NaN   NaN   NaN   0.000000      NaN    NaN  
2020-08-07          NaN        NaN   NaN   NaN   0.038462      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-05,0.001127,0.022849,0.016891,0.002300,0.043461,0.001658,0.031534,0.005910,0.027297,0.004193,...,0.002722,0.006839,0.030201,0.036852,0.055037,0.020944,3.157968e-15,0.004911,0.034181,0.024432
2020-08-06,0.001473,0.022207,0.017082,0.003812,0.053272,0.000829,0.032789,0.005898,0.023251,0.005110,...,0.002922,0.006857,0.024491,0.040422,0.050531,0.023216,1.578984e-15,0.003873,0.027202,0.026194
2020-08-07,0.002349,0.022324,0.016407,0.007732,0.045179,0.000415,0.032789,0.005033,0.022133,0.005805,...,0.002913,0.006084,0.025348,0.039240,0.051265,0.023699,7.894919e-16,0.009855,0.036074,0.026003


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-05,0.003074,0.030811,0.008877,8.707632e-18,0.040151,9.704881e-35,0.034970,0.005810,0.044738,0.000388,...,0.001706,0.00876,0.040144,0.039173,0.187404,0.024951,0.0,0.006685,0.017649,0.029645
2020-08-06,0.003083,0.031889,0.009197,4.353816e-18,0.036205,4.852441e-35,0.035142,0.004204,0.043938,0.000194,...,0.001380,0.00438,0.031768,0.037535,0.148652,0.029330,0.0,0.003343,0.074165,0.033341
2020-08-07,0.005008,0.018604,0.008133,2.176908e-18,0.041540,2.426220e-35,0.036390,0.005340,0.044525,0.000792,...,0.001744,0.00219,0.033027,0.033619,0.073741,0.025534,0.0,0.005608,0.039595,0.123813


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-05,0.002282,0.017663,0.020452,0.003430,0.027828,0.021798,0.029920,0.014263,0.014546,0.008110,...,0.000624,0.016855,0.051983,0.023153,0.007084,0.067281,1.850372e-17,0.018398,0.093434,0.052164
2020-08-06,0.003044,0.013955,0.019787,0.003533,0.027748,0.010899,0.029937,0.018574,0.015408,0.007446,...,0.000658,0.015000,0.039514,0.024914,0.017977,0.055475,9.251859e-18,0.011436,0.057216,0.036583
2020-08-07,0.002741,0.018388,0.019124,0.008409,0.036951,0.005450,0.032219,0.016449,0.027535,0.006121,...,0.000329,0.014914,0.039663,0.012457,0.012815,0.049672,4.625929e-18,0.010729,0.069915,0.050333


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-05,0.012279,0.019741,NaN,0.008402,0.018683,0.011255,0.010274,0.002130,0.008201,6.106084e-44,...,0.014892,0.020933,0.009871,0.003006,0.040665,0.010453,0.011822,0.015056,0.014329,0.013626
2020-08-06,0.016377,0.014946,NaN,0.008188,0.017280,0.015741,0.009711,0.001264,0.006356,3.053042e-44,...,0.017315,0.017721,0.011870,0.004637,0.041122,0.009530,0.012843,0.015626,0.014532,0.011945
2020-08-07,0.017035,0.015081,NaN,0.007897,0.019389,0.012928,0.009480,0.001378,0.005781,1.526521e-44,...,0.019108,0.016490,0.011294,0.003357,0.041519,0.015274,0.011921,0.019767,0.015824,0.013930


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-05      0.005453  0.017468  0.002644  0.014017  0.010554  0.006202   
2020-08-06      0.012435  0.017464  0.002208  0.044769  0.014673  0.009858   
2020-08-07      0.013360  0.018051  0.004644  0.026017  0.017218  0.009373   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-08-05      0.002911  0.025320  0.006430  0.018110  ...  0.031423   
2020-08-06      0.002782  0.026489  0.005018  0.027127  ...  0.031840   
2020-08-07      0.003375  0.022814  0.002509  0.023253  ...  0.031545   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-08-05      0.000008  0.026050  0.018515   0.006565  0.008675  0.007149   
2020-08-06      0.000004  0.044275 -0.003563   0.005267  0.009847  0.011037   
2020-08-07      0.000002  0.029491 -0.001781   0.010539  0.010373  0.009195   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-08-05           -1.0  0.064052  0.000245  
2020-08-06           -1.0  0.040647  0.000122  
2020-08-07           -1.0  0.105069  0.000061  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-05,0.016045,0.007079,NaN,0.017710,0.031100,0.014809,0.001503,0.000160,0.000985,NaN,...,0.021237,0.034330,0.017514,0.000140,0.000563,0.010661,0.005659,0.016320,0.008857,0.004886
2020-08-06,0.013627,0.003539,NaN,0.017756,0.022440,0.018263,0.001022,0.000080,0.000492,NaN,...,0.028975,0.032749,0.013344,0.008842,0.062782,0.010828,0.011758,0.008160,0.008552,0.002443
2020-08-07,0.012940,0.001770,NaN,0.018748,0.017045,0.014720,0.001861,0.000491,0.001098,NaN,...,0.022919,0.032596,0.014248,0.004421,0.031391,0.009329,0.011626,0.016177,0.010411,0.019740


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State  Alabama                                                        \
Admin2          Autauga   Baldwin   Barbour      Bibb    Blount       Bullock   
2020-08-05      0.02500  0.010142  0.001956  0.197917  0.007812  3.051770e-06   
2020-08-06      0.01250  0.005071  0.000978  0.098958  0.003906  1.525885e-06   
2020-08-07      0.00625  0.002535  0.000489  0.049479  0.001953  7.629426e-07   

Province_State                                              ... Wyoming  \
Admin2            Butler   Calhoun  Chambers      Cherokee  ...    Park   
2020-08-05      0.000031  0.129557  0.000038  1.019153e-14  ...     NaN   
2020-08-06      0.000016  0.064779  0.000019  5.095765e-15  ...     NaN   
2020-08-07      0.000008  0.032389  0.000010  2.547883e-15  ...     NaN   

Province_State                                                             \
Admin2         Platte Sheridan Sublette Sweetwater Teton Uinta Unassigned   
2020-08-05        NaN      NaN      NaN        NaN   NaN   NaN   0.005082   
2020-08-06        NaN      NaN      NaN        NaN   NaN   NaN   0.002541   
2020-08-07        NaN      NaN      NaN        NaN   NaN   NaN   0.020501   

Province_State                  
Admin2         Washakie Weston  
2020-08-05          NaN    NaN  
2020-08-06          NaN    NaN  
2020-08-07          NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200807,AK,4288,264563,NaN,37.0,NaN,NaN,NaN,3.0,...,268851,0,0,d608cee3e98d23a9e920faab4361b771c779880f,0,0,0,0,0,NaN
1,20200807,AL,98301,651115,NaN,1575.0,11537.0,NaN,1219.0,NaN,...,749416,21,224,b180d67ecb2784ceaaaf7fef67dcee5833fadd52,0,0,0,0,0,NaN
2,20200807,AR,48039,503484,NaN,514.0,3168.0,NaN,NaN,111.0,...,551523,6,50,1e9a8ed04dc081a895e26f03a10711ce88d78be2,0,0,0,0,0,NaN
3,20200807,AS,0,1396,NaN,NaN,NaN,NaN,NaN,NaN,...,1396,0,0,9395052fa25b89c3561068bed6cc6c425623bb9c,0,0,0,0,0,NaN
4,20200807,AZ,185053,819792,NaN,1772.0,19241.0,565.0,NaN,411.0,...,1004845,79,5173,73a278d71ae7e87e4e634538a146339f015d5531,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200807,AK,4288.0,264563.0,NaN,37.0,NaN,NaN,NaN,3.0,...,268851,0,0,d608cee3e98d23a9e920faab4361b771c779880f,0,0,0,0,0,NaN
1,20200807,AL,98301.0,651115.0,NaN,1575.0,11537.0,NaN,1219.0,NaN,...,749416,21,224,b180d67ecb2784ceaaaf7fef67dcee5833fadd52,0,0,0,0,0,NaN
2,20200807,AR,48039.0,503484.0,NaN,514.0,3168.0,NaN,NaN,111.0,...,551523,6,50,1e9a8ed04dc081a895e26f03a10711ce88d78be2,0,0,0,0,0,NaN
3,20200807,AS,0.0,1396.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1396,0,0,9395052fa25b89c3561068bed6cc6c425623bb9c,0,0,0,0,0,NaN
4,20200807,AZ,185053.0,819792.0,NaN,1772.0,19241.0,565.0,NaN,411.0,...,1004845,79,5173,73a278d71ae7e87e4e634538a146339f015d5531,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-07,AK,4288,264563,NaN,37.0,NaN,NaN,NaN,3.0,NaN,...,268851,0,0,d608cee3e98d23a9e920faab4361b771c779880f,0,0,0,0,0,NaN
2020-08-07,AL,98301,651115,NaN,1575.0,11537.0,NaN,1219.0,NaN,658.0,...,749416,21,224,b180d67ecb2784ceaaaf7fef67dcee5833fadd52,0,0,0,0,0,NaN
2020-08-07,AR,48039,503484,NaN,514.0,3168.0,NaN,NaN,111.0,429.0,...,551523,6,50,1e9a8ed04dc081a895e26f03a10711ce88d78be2,0,0,0,0,0,NaN
2020-08-07,AS,0,1396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1396,0,0,9395052fa25b89c3561068bed6cc6c425623bb9c,0,0,0,0,0,NaN
2020-08-07,AZ,185053,819792,NaN,1772.0,19241.0,565.0,NaN,411.0,NaN,...,1004845,79,5173,73a278d71ae7e87e4e634538a146339f015d5531,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-07,AK,4288.0,264563.0,NaN,37.0,NaN,NaN,NaN,3.0,NaN,...,268851,0,0,d608cee3e98d23a9e920faab4361b771c779880f,0,0,0,0,0,NaN
2020-08-07,AL,98301.0,651115.0,NaN,1575.0,11537.0,NaN,1219.0,NaN,658.0,...,749416,21,224,b180d67ecb2784ceaaaf7fef67dcee5833fadd52,0,0,0,0,0,NaN
2020-08-07,AR,48039.0,503484.0,NaN,514.0,3168.0,NaN,NaN,111.0,429.0,...,551523,6,50,1e9a8ed04dc081a895e26f03a10711ce88d78be2,0,0,0,0,0,NaN
2020-08-07,AS,0.0,1396.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1396,0,0,9395052fa25b89c3561068bed6cc6c425623bb9c,0,0,0,0,0,NaN
2020-08-07,AZ,185053.0,819792.0,NaN,1772.0,19241.0,565.0,NaN,411.0,NaN,...,1004845,79,5173,73a278d71ae7e87e4e634538a146339f015d5531,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE